### Lab 1: API Interactions and Spatial Joins

#### Goals:
1. Interact with NDAWN data, CKAN API, and ArcGIS REST API.
2. Transform the data into spatially consistent coordinate systems.
3. Merge the data into a single geodatabase.

#### Libraries and Databases

In [2]:
from arcgis.gis import GIS
gis = GIS("home")

In [16]:
# Import Libraries
import io
import os
import sys
import arcpy
from zipfile import ZipFile
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureCollection, FeatureSet
from arcgis.geometry import SpatialReference

#### NDAWN API

In [4]:
# Request: NDAWN Data
ndawn_link = r"https://ndawn.ndsu.nodak.edu/table.csv?station=219&station=220&station=223&station=93&station=183&station=156&station=70&variable=wdmxt&variable=wdsr&variable=wdr&variable=wddp&ttype=weekly&quick_pick=&begin_date=2024-09-16&count=1"
ndawn_request = requests.get(ndawn_link)
ndawn_response = ndawn_request.content

#print(ndawn_response)
df = pd.read_csv(io.StringIO(ndawn_response.decode('utf-8')), skiprows=[0,1,2,4], index_col=False)
df.drop(df.columns[[8,9,11,12,14,15,17,18]], axis=1, inplace=True)
df.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Max Temp,Total Solar Rad,Total Rainfall,Avg Dew Point
0,Crookston,47.823333,-96.620556,910,2024,9,16,79.595,325.285,0.032,56.351
1,Fertile,47.590146,-96.308661,1127,2024,9,16,78.543,337.485,0.126,56.754
2,Fountain,43.745483,-92.186039,1181,2024,9,16,81.197,340.412,1.279,58.559
3,Fox,48.877750,-95.850170,1040,2024,9,16,73.382,289.654,1.625,56.231
4,Foxhome,46.268190,-96.215630,1096,2024,9,16,79.929,377.351,0.030,56.799


In [5]:
# Convert DataFrame to SeDF
sedf_ndawn = pd.DataFrame.spatial.from_xy(df, "Longitude", "Latitude", crs=4326)
sedf_ndawn.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Max Temp,Total Solar Rad,Total Rainfall,Avg Dew Point,SHAPE
0,Crookston,47.823333,-96.620556,910,2024,9,16,79.595,325.285,0.032,56.351,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Fertile,47.590146,-96.308661,1127,2024,9,16,78.543,337.485,0.126,56.754,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
2,Fountain,43.745483,-92.186039,1181,2024,9,16,81.197,340.412,1.279,58.559,"{""spatialReference"": {""wkid"": 4326}, ""x"": -92...."
3,Fox,48.877750,-95.850170,1040,2024,9,16,73.382,289.654,1.625,56.231,"{""spatialReference"": {""wkid"": 4326}, ""x"": -95...."
4,Foxhome,46.268190,-96.215630,1096,2024,9,16,79.929,377.351,0.030,56.799,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."


In [6]:
# Save the SpatialDataFrame to a shapefile
output_file_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn.shp"
sedf_ndawn.spatial.to_featureclass(output_file_path)

# Example of projecting a shapefile to a different CRS
output_shapefile = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn_proj.shp"
spatial_reference = arcpy.SpatialReference(4326)

# Project the Shapefile
arcpy.Project_management(output_file_path, output_shapefile, spatial_reference)

# Shapefile back into a Spatial DataFrame
sedf_ndawn_proj = pd.DataFrame.spatial.from_featureclass(output_shapefile)

In [7]:
# Display the first few rows of the projected Spatial DataFrame
sedf_ndawn_proj.head()

,FID,Id,station_na,latitude,longitude,elevation,year,month,day,avg_max_te,total_sola,total_rain,avg_dew_po,SHAPE
0,0,0,,47.823333,-96.620556,910,2024,9,16,0,0,0,0,"{""x"": -96.620556, ""y"": 47.823333, ""spatialRefe..."
1,1,0,,47.590146,-96.308661,1127,2024,9,16,0,0,0,0,"{""x"": -96.308661, ""y"": 47.590146, ""spatialRefe..."
2,2,0,,43.745483,-92.186039,1181,2024,9,16,0,0,0,0,"{""x"": -92.186039, ""y"": 43.745483, ""spatialRefe..."
3,3,0,,48.87775,-95.85017,1040,2024,9,16,0,0,0,0,"{""x"": -95.85017, ""y"": 48.87775, ""spatialRefere..."
4,4,0,,46.26819,-96.21563,1096,2024,9,16,0,0,0,0,"{""x"": -96.21563, ""y"": 46.26819, ""spatialRefere..."


#### CKAN API

In [9]:
# Request: MN GeoCommons -- MN County Boundaries

# Call CKAN API and Search for Minnesota Counties JSON 
mn_meta = r"https://gisdata.mn.gov/api/3/action/package_search?q=minnesota%20counties"
mn_meta_response = requests.get(mn_meta)

mn_meta_output = mn_meta_response.json()

# Scan JSON for MN County GeoDatabase Zip File
mn_county_zip = mn_meta_output["result"]["results"][0]["resources"][3]["url"]
print(mn_county_zip)

https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip


In [12]:
import zipfile

# Save the ZIP file
mn_county_response = requests.get(mn_county_zip)

zip_file_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\mn_county_bounds.zip"
with open(zip_file_path, 'wb') as f:
    f.write(mn_county_response.content)  # Write the response content to the ZIP file

# Open the ZIP file and print its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Print the contents of the ZIP file
    zip_ref.printdir()   # This will show the files inside the ZIP
    
    # Create a directory for extraction if it doesn't exist
    extraction_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\extracted_files"
    os.makedirs(extraction_path, exist_ok=True)  # Create the directory if it doesn't exist
    
    # Extract all files to the specified directory
    zip_ref.extractall(extraction_path)

# Show the files you just extracted
print("Files in the extracted directory:")
print(os.listdir(extraction_path))

File Name                                             Modified             Size
mn_county_boundaries_500.prj                   2013-09-13 11:45:32          424
mn_county_boundaries_multipart.shp.xml         2013-09-13 11:45:44        19830
mn_county_boundaries_500.sbx                   2015-07-31 16:38:02          212
mn_county_boundaries_1500.shx                  2015-07-31 16:38:16         1012
mn_county_boundaries_1000.shx                  2015-07-31 16:38:16         1012
mn_county_boundaries_1500.dbf                  2015-07-31 16:38:16        21508
mn_county_boundaries_multipart.dbf             2015-07-31 16:38:16         5904
mn_county_boundaries_1000.prj                  2013-09-13 11:45:22          424
mn_county_boundaries_anno.sbn                  2013-09-13 11:45:38         7676
mn_county_boundaries_1500.shp.xml              2013-09-13 11:45:28        19385
mn_county_boundaries.dbf                       2015-07-31 16:38:16        17553
mn_county_boundaries_500.shp            

In [13]:
# Read the shapefile
sedf_cty = pd.DataFrame.spatial.from_featureclass(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\extracted_files\mn_county_boundaries.shp")
sedf_cty.spatial.project(SpatialReference(4326))
sedf_cty.head()

,FID,AREA,PERIMETER,CTYONLY_,CTYONLY_ID,COUN,CTY_NAME,CTY_ABBR,CTY_FIPS,Shape_Leng,Shape_Area,SHAPE
0,0,4608320923.5,388250.145723,2,1,39,Lake of the Woods,LOTW,77,388250.145362,4608320923.18,"{""rings"": [[[-95.32315990924144, 48.9989365543..."
1,1,2862183702.23,263017.482774,3,2,35,Kittson,KITT,69,263017.482649,2862183701.75,"{""rings"": [[[-96.40549738325757, 49.0000809414..."
2,2,4347098503.13,302590.75293,4,3,68,Roseau,ROSE,135,302590.752705,4347098503.06,"{""rings"": [[[-96.40549738325757, 49.0000809414..."
3,3,8167237870.92,412897.435943,5,4,36,Koochiching,KOOC,71,412897.435786,8167237869.65,"{""rings"": [[[-94.42997413261082, 48.7011535351..."
4,4,4698732287.55,374207.651291,6,5,45,Marshall,MARS,89,374207.651278,4698732287.94,"{""rings"": [[[-96.38822010063237, 48.5441549753..."


In [14]:
# Optional: Save the SpatialDataFrame to a shapefile
output_file_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_cty.shp"
sedf_cty.spatial.to_featureclass(output_file_path)

# Display the first few rows of the SpatialDataFrame
sedf_cty.head()

The Field type is invalid or unsupported for the operation. [ctyonly_]


,FID,AREA,PERIMETER,CTYONLY_,CTYONLY_ID,COUN,CTY_NAME,CTY_ABBR,CTY_FIPS,Shape_Leng,Shape_Area,SHAPE
0,0,4608320923.5,388250.145723,2,1,39,Lake of the Woods,LOTW,77,388250.145362,4608320923.18,"{""rings"": [[[-95.32315990924144, 48.9989365543..."
1,1,2862183702.23,263017.482774,3,2,35,Kittson,KITT,69,263017.482649,2862183701.75,"{""rings"": [[[-96.40549738325757, 49.0000809414..."
2,2,4347098503.13,302590.75293,4,3,68,Roseau,ROSE,135,302590.752705,4347098503.06,"{""rings"": [[[-96.40549738325757, 49.0000809414..."
3,3,8167237870.92,412897.435943,5,4,36,Koochiching,KOOC,71,412897.435786,8167237869.65,"{""rings"": [[[-94.42997413261082, 48.7011535351..."
4,4,4698732287.55,374207.651291,6,5,45,Marshall,MARS,89,374207.651278,4698732287.94,"{""rings"": [[[-96.38822010063237, 48.5441549753..."


#### ArcGIS REST API

In [17]:
# ArcGIS RestAPI Access
import requests


arc_api_link = r"https://arcgis.dnr.state.mn.us/host/rest/services/Hosted/DOT_Roads/FeatureServer/1/query?where=1%3D1&outFields=*&f=geojson"
arc_response = requests.get(arc_api_link, stream=True).json()

arc_geojson = FeatureSet.from_dict(arc_response)

# Create a SeDF from the GeoJSON data
sedf_streets = arc_geojson.sdf
sedf_streets.head()

,OBJECTID,SHAPE
0,1,"{""paths"": [[[-93.49663828095794, 43.8960325529..."
1,2,"{""paths"": [[[-93.62792962458286, 44.9062905066..."
2,3,"{""paths"": [[[-94.02933652101696, 45.2683112715..."
3,4,"{""paths"": [[[-96.20515046107992, 46.2821867213..."
4,5,"{""paths"": [[[-95.59511667768729, 43.6183018564..."


In [18]:
# Save the SpatialDataFrame to a shapefile
output_file_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_streets.shp"
sedf_streets.spatial.to_featureclass(output_file_path)

# Example of projecting a shapefile to a different CRS
output_shapefile = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_streets_proj.shp"
spatial_reference = arcpy.SpatialReference(4326)

# Project the Shapefile
arcpy.Project_management(output_file_path, output_shapefile, spatial_reference)

# Shapefile back into a Spatial DataFrame
sedf_streets_proj = pd.DataFrame.spatial.from_featureclass(output_shapefile)

In [19]:
# Display the first few rows of the SpatialDataFrame
sedf_streets_proj.head()

,FID,Id,objectid,SHAPE
0,0,0,1,"{""paths"": [[[-93.49663828099995, 43.8960325530..."
1,1,0,2,"{""paths"": [[[-93.62792962499998, 44.9062905070..."
2,2,0,3,"{""paths"": [[[-94.02933652099995, 45.2683112720..."
3,3,0,4,"{""paths"": [[[-96.20515046099996, 46.2821867210..."
4,4,0,5,"{""paths"": [[[-95.59511667799995, 43.6183018560..."


### Transform the Data

Look for the counties that contain the study site cities. Then, use these counties to identify the streets that are within these counties.

#### Check the data types and spatial references of the datasets

In [20]:
# Geometry Check:
desc_counties = arcpy.Describe(counties_shp)
desc_cities = arcpy.Describe(cities_shp)
desc_roads = arcpy.Describe(roads_shp)

print(f"Counties Geometry Type: {desc_counties.shapeType}")
print(f"Cities Geometry Type: {desc_cities.shapeType}")
print(f"Roads Geometry Type: {desc_roads.shapeType}")

NameError: name 'counties_shp' is not defined

In [ ]:
# Coordinate System, Spatial Reference Check:
print(f"Counties Spatial Reference: {desc_counties.spatialReference.name}")
print(f"Cities Spatial Reference: {desc_cities.spatialReference.name}")
print(f"Roads Spatial Reference: {desc_roads.spatialReference.name}")

#### Spatially Join the Data Layers without Analysis

In [21]:
# Spatially Join all 3 Layers

# First Spatial Join Layer
target_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_cty.shp"
join_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn_proj.shp"
out_feature_class_1 = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_test1.shp"
out_feature_class_2 = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_test2.shp"

arcpy.SpatialJoin_analysis(target_features, join_features, out_feature_class_1)

# Second Spatial Join Layer
additional_join_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_streets_proj.shp"
arcpy.SpatialJoin_analysis(out_feature_class_1, additional_join_features, out_feature_class_2)

<Result 'C:\\Users\\ethan\\Desktop\\ARLT_MGIS\\GIS5571\\Lab1\\Joined_Files\\join_test2.shp'>

In [22]:
# Read the shapefile
sedf_test2 = pd.DataFrame.spatial.from_featureclass(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_test2.shp")
sedf_test2.spatial.project(SpatialReference(4326))
sedf_test2.head()

,FID,Join_Count,TARGET_FID,Join_Cou_1,TARGET_F_1,Id,area,perimeter,shape_area,Id_1,station_na,latitude,longitude,elevation,year,month,day,avg_max_te,total_sola,total_rain,avg_dew_po,Id_12,objectid,SHAPE
0,0,0,0,3,0,0,4608320923.5,388250.145723,4608320923.18,0,,0.0,0.0,0,0,0,0,0,0,0,0,0,363,"{""rings"": [[[-95.32315990899997, 48.9989365540..."
1,1,0,1,4,1,0,2862183702.23,263017.482774,2862183701.75,0,,0.0,0.0,0,0,0,0,0,0,0,0,0,178,"{""rings"": [[[-96.40549738299995, 49.0000809410..."
2,2,2,2,6,2,0,4347098503.13,302590.75293,4347098503.06,0,,48.87775,-95.85017,1040,2024,9,16,0,0,0,0,0,186,"{""rings"": [[[-96.40549738299995, 49.0000809410..."
3,3,0,3,6,3,0,8167237870.92,412897.435943,8167237869.65,0,,0.0,0.0,0,0,0,0,0,0,0,0,0,49,"{""rings"": [[[-94.42997413299997, 48.7011535350..."
4,4,0,4,9,4,0,4698732287.55,374207.651291,4698732287.94,0,,0.0,0.0,0,0,0,0,0,0,0,0,0,47,"{""rings"": [[[-96.38822010099994, 48.5441549750..."


#### Spatial Join as Analysis: Locational Intersection of Data

In [23]:
# Find the counties that contain the NDAWN Data
counties_shp = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_cty.shp"
cities_shp = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn_proj.shp"
roads_shp = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_streets_proj.shp" 

# Spatial Join to find counties with cities
counties_with_cities = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\counties_with_cities.shp"
arcpy.SpatialJoin_analysis(counties_shp, cities_shp, counties_with_cities, 
                           join_type="KEEP_COMMON", 
                           match_option="INTERSECT")

print(f"Counties with cities saved to {counties_with_cities}")

Counties with cities saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\counties_with_cities.shp


In [24]:
# Spatial Join to find roads within the filtered counties
roads_within_counties = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\roads_within_counties.shp"
arcpy.SpatialJoin_analysis(roads_shp, counties_with_cities, roads_within_counties, 
                           join_type="KEEP_COMMON", 
                           match_option="INTERSECT")

print(f"Roads within counties saved to {roads_within_counties}")


Roads within counties saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\roads_within_counties.shp


### Export the Data as a Geodatabase File

#### SEDF_Test2 to a Geodatabase File

In [25]:
# Path where you want to create the new geodatabase
gdb_test2_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\test2_geodatabase.gdb"

# Create a new file geodatabase
arcpy.CreateFileGDB_management(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1", "test2_geodatabase")

print(f"Geodatabase created at {gdb_test2_path}")

Geodatabase created at C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\test2_geodatabase.gdb


In [26]:
# Spatial DataFrame Test2 shapefile path
sdf_Test2 = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_test2.shp"

# Specify the output feature class name
output_fc_test2 = "sedf_test2"

# Convert the shapefile to a feature class in the geodatabase
arcpy.FeatureClassToGeodatabase_conversion([sdf_Test2], gdb_test2_path)

print(f"Feature class {output_fc_test2} saved to {gdb_test2_path}")

Feature class sedf_test2 saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\test2_geodatabase.gdb


#### Analysized and Filtered Layers to Geodatabase

In [27]:
# Re-Define the filepaths:
cities_shp = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn_proj.shp"
counties_with_cities = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\counties_with_cities.shp"
roads_within_counties = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\roads_within_counties.shp"

# Spatially Join all 3 Layers

# First Spatial Join Layer
target_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\counties_with_cities.shp"
join_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\roads_within_counties.shp"
out_fc_cty_roads = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_cty_roads.shp"
out_fc_all = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_all.shp"

arcpy.SpatialJoin_analysis(target_features, join_features, out_fc_cty_roads)

# Second Spatial Join Layer
additional_join_features = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\sedf_ndawn_proj.shp"
arcpy.SpatialJoin_analysis(out_fc_cty_roads, additional_join_features, out_fc_all)

<Result 'C:\\Users\\ethan\\Desktop\\ARLT_MGIS\\GIS5571\\Lab1\\Joined_Files\\join_all.shp'>

In [28]:
# Path where you want to create the new geodatabase
gdb_joinAll_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\joinAll_geodatabase.gdb"

# Create a new file geodatabase
arcpy.CreateFileGDB_management(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1", "joinAll_geodatabase")

print(f"Geodatabase created at {gdb_joinAll_path}")

Geodatabase created at C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\joinAll_geodatabase.gdb


In [29]:
# Spatial DataFrame JoinAll shapefile path
sdf_joinAll = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\Joined_Files\join_all.shp"

# Specify the output feature class name
output_fc_joinAll = "sedf_joinAll"

# Convert the shapefile to a feature class in the geodatabase
arcpy.FeatureClassToGeodatabase_conversion([sdf_joinAll], gdb_joinAll_path)

print(f"Feature class {output_fc_joinAll} saved to {gdb_joinAll_path}")

Feature class sedf_joinAll saved to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab1\joinAll_geodatabase.gdb
